In [71]:
import numpy as np
import re
import pandas as pd
import copy
from pathlib import Path
import requests
import json
from pathlib import Path
path = Path.cwd().parent
raw_data = str(path) + '/raw_data/'
data = str(path) + '/data/'

In [72]:
date = '20221212'


#### 향후 수정사항

In [73]:
#### 향후 수정사항
def slicing(df,i) :
    pattern = '.*동'
    reg = re.search(pattern, df['장비명'][i])
    reg = reg.group()
    df['주차장'][i] = reg
    return reg


def change(x):
    if x =='13-1동' or x =='13-2동' or x =='13-3동':
        return '13동'
    elif x == '14-1동' or x == '14-2동':
        return '14동'
    else :
        return x
def parking(x):
    if x =='15동':
        return 15
    elif x == '14동':
        return 14
    elif x == '13동':
        return 13
    elif x == '12동':
        return 12



def confusion(df):
    if df['비율'] <= 0.3 :
        return 0
    elif df['비율'] <= 0.5 :
        return 1
    elif df['비율'] <= 0.8 :
        return 2
    else :
        return 3

# 데이터 Load

In [74]:
darae_in = pd.read_csv(raw_data+ 'VW_InCarinfo (차량 입차 정보 조회).csv')
darae_in = darae_in.dropna()
darae_out = pd.read_csv(raw_data+'VW_OutCarinfo (차량 출차 정보 조회).csv')
whether = pd.read_csv(data +'Whether.csv', encoding='euc-kr', parse_dates=['일시'], index_col='일시')


/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_84276/1529426654.py:1: DtypeWarning: Columns (0,1,2,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  darae_in = pd.read_csv(raw_data+ 'VW_InCarinfo (차량 입차 정보 조회).csv')


# 전처리(학습데이터)

In [75]:
darae_in[['입차고객구분']]= darae_in[['입차고객구분']].apply(lambda x: x.str.strip(), axis = 1)
darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']] = darae_in[['장비명','입차구분','입차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)
darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']]= darae_out[['장비명','출차구분','출차고객구분','출차여부','차량종류']].apply(lambda x: x.str.strip(), axis = 1)

darae_in['주차장'] = 0
for i in range(len(darae_in)):
    slicing(darae_in,i)
darae_out['주차장'] = 0

for i in range(len(darae_out)):
    slicing(darae_out,i)

darae_in['주차장'] = darae_in['주차장'].apply(change)
darae_out['주차장'] = darae_out['주차장'].apply(change)
test_in = copy.deepcopy(darae_in)
test_in.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_in['입출차여부'] = '입차'
test_out = copy.deepcopy(darae_out)
test_out.columns = ['일자','시각','차량번호','주차일련번호','입출차카드번호','장비명','입출차구분','입출차고객구분','출차여부','차량종류','주차장']
test_out['입출차여부'] = '출차'


df = pd.concat([test_in, test_out]).sort_values(by=['차량번호','일자','시각'])#[['차량번호','입출차여부','일자','시각','입출차구분','출차여부','장비명']]
import datetime
df['입출차시각'] = df['일자'] + df['시각']
df['입출차시각'] = pd.to_datetime(df['입출차시각'], format= '%Y-%m-%d %H:%M:%S', errors='raise')
df['시'] = df['입출차시각'].dt.hour
df.reset_index(inplace = True)
car_arr = np.array(df[['일자','시각','차량번호']])
test_idx_list =[]

for idx in range(len(car_arr)-1):
    if car_arr[idx][0] == car_arr[idx+1][0] and car_arr[idx][1] == car_arr[idx+1][1] and car_arr[idx][2] == car_arr[idx+1][2]:
        test_idx_list.append(idx)
        test_idx_list.append(idx+1)
        #test_idx_list.append(idx+2)
test_idx_list = list(set(test_idx_list))
test_df = df.loc[test_idx_list]
eleminate_index = test_df[test_df['입출차여부']=='출차'].index
df2 = df.drop(eleminate_index)
df2.drop(['index'],axis = 1 , inplace = True)
df2.reset_index(inplace=True)
df2.drop(['index'],axis = 1 , inplace = True)



car_arr = np.array(df2[['차량번호','입출차여부','시각']])
idx_list = []
for idx in range(len(car_arr)):
    if idx == 0 :
        idx_list.append(idx)
    else :  # 차량 번호가 같고 상태가 다른 경우(입차 > 출차 or 출차 > 입차)
        if (car_arr[idx][0] == car_arr[idx-1][0]) and (car_arr[idx][1]!=car_arr[idx-1][1]):
            idx_list.append(idx)

            #차량번호가 바뀌는 경우
        elif car_arr[idx][0] != car_arr[idx-1][0]:
            idx_list.append(idx)


process_df = df2.loc[idx_list]
process_df = df2.loc[idx_list]
process_df['입출차여부'].value_counts()

process_df['연'] = process_df['입출차시각'].dt.year
process_df['월'] = process_df['입출차시각'].dt.month
process_df['일'] = process_df['입출차시각'].dt.day
process_df['시'] = process_df['입출차시각'].dt.hour
process_df['분'] = process_df['입출차시각'].dt.minute
process_df['초'] = process_df['입출차시각'].dt.second
process_df = process_df[(process_df['시']>=7) & (process_df['시']<=9)]

parking_info = pd.read_excel(data_path + '/raw_data/주차장면수.xlsx')
dataset = pd.merge(process_df,parking_info[['장소','합계']], left_on='주차장', right_on='장소')

dataset = dataset[['입출차시각','연','월','일','시','분','초','입출차여부','주차장','합계']]
dataset = dataset.sort_values(by = ['주차장','입출차시각'])
dataset['잔류차량수'] = 0
arr = np.array(dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']])

for idx in range(len(arr)):
    if idx == 0 :
        if arr[idx][5] == '입차' :
            arr[idx][7] = arr[idx][7] +1
        else :
            arr[idx][7] = arr[idx][7] -1
    else :
        pre_time = list(arr[idx-1][[0,1,2,6]])
        now_time = list(arr[idx][[0,1,2,6]])
        status =  arr[idx][5]
            # 이전 데이터와 연,월,일 주차장이 같은 경우
        if pre_time == now_time :
            #print('>')
            if status == '입차' :
                arr[idx][7] = arr[idx-1][7] +1
            else :
                arr[idx][7] = arr[idx-1][7] -1

            #이전 데이터와 연,월,일 주차장이 다른 경우 > 0부터 시작함
        else :
            if status == '입차':
                arr[idx][7] =1
            else :
                arr[idx][7] = -1

dataset[['연','월','일','시','분','입출차여부','주차장','잔류차량수']] = arr

dataset['비율']=dataset['잔류차량수']/dataset['합계']


dataset['혼잡도']=dataset.apply(confusion,axis=1)
# 날씨데이터 붙이기
# whether csv read, 일시를 index로 넣고 datetime 형태로 변환함
# 강수량, 적설이 null인 경우 모두 0으로 바꿈
whether = whether.fillna(0)
# 일시의 freq을 hour로 변경
whether.index.freq = 'h'

'''
데이터가 누락된 시간을 확인하기 위한 부분 (파일로 받은 경우)
API로 받는 경우 날짜는 있지만 모든 데이터가 Null값이기 때문에 해당 조건으로 수정해야 할 것으로 보임

j = whether.index[0]
for i in whether.index:
    if i != j + pd.Timedelta("1 hour"):
        print(j, i)
    j = i
'''
# 지점코드, 지점명을 없앰
whether = whether.iloc[:, [2,3,4,5,6]]


whether = whether.reset_index()
whether['연'] = whether['일시'].dt.year
whether['월'] = whether['일시'].dt.month
whether['일'] = whether['일시'].dt.day
whether['시'] = whether['일시'].dt.hour
dataset = pd.merge(dataset,whether, on =['연','월','일','시'], how= 'left')



import holidays
kr_holidays = holidays.KR()
def holiday_check(x):
    if x in kr_holidays:
        return 1
    else :
        return 0


dataset['공휴일'] = dataset['입출차시각'].apply(holiday_check)
dataset['요일'] = dataset['입출차시각'].dt.weekday
dataset = dataset[['입출차시각', '연', '월', '일', '시', '분','초','공휴일','요일','주차장',
         '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '적설(cm)',
         '잔류차량수', '비율','혼잡도']]
# 날짜 매칭이 안되는 데이터들 제거
dataset = dataset.dropna()
dataset['주차장'] = dataset['주차장'].apply(parking)


dataset.drop('입출차시각',axis =1,inplace= True)
dataset.drop('초',axis =1,inplace= True)
#dataset.reset_index(inplace=True)



/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_84276/1382307645.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_84276/1382307645.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['주차장'][i] = reg


In [76]:
#trend 만들기(30분 간격)
def tmp(df):
    if df['분'] <= 29:
        return 0
    else :
        return 1

dataset['tmp'] = dataset.apply(tmp, axis =1)
dataframe =dataset.groupby(['연','월','일','시','tmp']).count().reset_index()[['연','월','일','시','tmp']]
dataframe['trend'] = range(0, len(dataframe))
dataset = pd.merge(dataset,dataframe, on = ['연','월','일','시','tmp'],how = 'left')
dataset.drop('tmp',axis =1,inplace= True)

dataset.index = dataset['trend']
dataset.drop('trend', axis=1, inplace=True)
dataset.reset_index(inplace=True)
dataset.head()

,trend,연,월,일,시,분,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm),잔류차량수,비율,혼잡도
0,0,2022,9,1,7,7,0,3,12,20.2,0.0,0.9,94.0,0.0,1,0.012195,0
1,0,2022,9,1,7,17,0,3,12,20.2,0.0,0.9,94.0,0.0,2,0.02439,0
2,0,2022,9,1,7,18,0,3,12,20.2,0.0,0.9,94.0,0.0,3,0.036585,0
3,1,2022,9,1,7,35,0,3,12,20.2,0.0,0.9,94.0,0.0,4,0.04878,0
4,1,2022,9,1,7,35,0,3,12,20.2,0.0,0.9,94.0,0.0,3,0.036585,0


In [77]:
dataset.to_csv(data + 'dataset.csv')

# 전처리(예측데이터)

In [78]:
weather_url_ = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
key = "oySL2JeGKZYtKFd4BsdWRR0lL1SbPvxVz2Gen1LrmPi6aKoPTx5YYujZz8dhRYRqmmLXAza2VNdxJGKE6DfMnw=="
params = {"servicekey":key,"pageNo":"1","numOfRows":"1000","dataType":"JSON","base_date":date,"base_time":"0500","nx":"66","ny":"103"}

response = requests.get(weather_url_, params=params)
content = json.loads(response.text)

df_whether = pd.DataFrame(content["response"]["body"]["items"]["item"])

df_tmp = df_whether[df_whether["category"]=="TMP"][["fcstDate","fcstTime","fcstValue"]].rename(columns={"fcstDate":"일시","fcstTime":"시간","fcstValue":"기온(°C)"})
df_pcp = df_whether[df_whether["category"]=="PCP"][["fcstDate","fcstTime","fcstValue"]].rename(columns={"fcstDate":"일시","fcstTime":"시간","fcstValue":"강수량(mm)"})
df_wsd = df_whether[df_whether["category"]=="WSD"][["fcstDate","fcstTime","fcstValue"]].rename(columns={"fcstDate":"일시","fcstTime":"시간","fcstValue":"풍속(m/s)"})
df_reh = df_whether[df_whether["category"]=="REH"][["fcstDate","fcstTime","fcstValue"]].rename(columns={"fcstDate":"일시","fcstTime":"시간","fcstValue":"습도(%)"})
df_sno = df_whether[df_whether["category"]=="SNO"][["fcstDate","fcstTime","fcstValue"]].rename(columns={"fcstDate":"일시","fcstTime":"시간","fcstValue":"적설(cm)"})

df_pcp.loc[df_pcp["강수량(mm)"]=="강수없음","강수량(mm)"] = 0
df_sno.loc[df_sno["적설(cm)"]=="적설없음","적설(cm)"] = 0

final_df = df_tmp.copy()
final_df = pd.merge(final_df,df_pcp,on=["일시","시간"],how="left")
final_df = pd.merge(final_df,df_wsd,on=["일시","시간"],how="left")
final_df = pd.merge(final_df,df_reh,on=["일시","시간"],how="left")
final_df = pd.merge(final_df,df_sno,on=["일시","시간"],how="left")

for ind in range(len(final_df)):
    final_df.loc[ind, "일시"] = datetime.datetime.strptime(final_df.loc[ind,"일시"][:4]+"-"+final_df.loc[ind,"일시"][4:6]+"-"+final_df.loc[ind,"일시"][6:]+" "+final_df.loc[ind,"시간"][:2],"%Y-%m-%d %H")

whether = final_df.drop("시간",axis=1)

whether['일시']= pd.to_datetime(whether['일시'])
whether['연'] = whether['일시'].dt.year
whether['월'] = whether['일시'].dt.month
whether['일'] = whether['일시'].dt.day
whether['시'] = whether['일시'].dt.hour
whether=whether[(whether['시']>=7) & (whether['시']<=10)]

In [79]:
whether['분'] = 0
whether2 = copy.deepcopy(whether)
whether2['분'] = 30
parking_list = dataset['주차장'].unique()

predict_dataset=  pd.DataFrame()
for parking in parking_list:
    predict_df = pd.concat([whether,whether2])
    predict_df['주차장']=parking
    predict_df = predict_df.sort_values(by=['일','시','분'])
    predict_df['공휴일'] = predict_df['일시'].apply(holiday_check)
    predict_df['요일'] = predict_df['일시'].dt.weekday
    predict_df['trend'] = range(dataset['trend'].max()+1, dataset['trend'].max() +1+ len(predict_df) )
    predict_dataset = pd.concat([predict_dataset,predict_df])
predict_dataset.drop('일시',axis =1,inplace= True)
predict_dataset = predict_dataset[dataset.iloc[:,:14].columns]
predict_dataset

,trend,연,월,일,시,분,공휴일,요일,주차장,기온(°C),강수량(mm),풍속(m/s),습도(%),적설(cm)
1,173,2022,12,12,7,0,0,0,12,-4,0,1.4,85,0
1,174,2022,12,12,7,30,0,0,12,-4,0,1.4,85,0
2,175,2022,12,12,8,0,0,0,12,-3,0,0.9,80,0
2,176,2022,12,12,8,30,0,0,12,-3,0,0.9,80,0
3,177,2022,12,12,9,0,0,0,12,-2,0,0.8,70,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,192,2022,12,14,8,30,0,2,15,-9,0,0.9,65,0
51,193,2022,12,14,9,0,0,2,15,-8,0,1,65,0
51,194,2022,12,14,9,30,0,2,15,-8,0,1,65,0
52,195,2022,12,14,10,0,0,2,15,-7,0,1.3,60,0


In [80]:
predict_dataset.to_csv(data +'predict_dataset.csv' )